### 📌 Goal: Compare performance across vector DBs: Chroma, FAISS, Qdrant, Weaviate

In [1]:
from rag_utils.chunking import chunk_texts
from rag_utils.embeddings import get_embeddings
from rag_utils.ingestion import extract_text
from rag_utils.indexing import (
    get_chroma_collection,
    get_faiss_index,
    get_weaviate_collection_local,
    get_qdrant_collection_local,
    add_to_chroma,
    add_to_faiss,
    add_to_qdrant_local,
    add_to_weaviate_local,
)
from config import settings
import os

In [ ]:
ingestion_folder = settings['paths']['ingestion_folder']
sample_file_path = os.path.join(ingestion_folder, 'machine_learning.txt')
data_folder = settings['paths']['data_folder']
text = extract_text(sample_file_path)

In [3]:
chunks = chunk_texts([text])
texts = [d for d in chunks]
embs = get_embeddings(texts)

In [ ]:
chroma = get_chroma_collection(os.path.join(ingestion_folder, "chroma_store"))
add_to_chroma(chroma, texts, embs)

faiss = get_faiss_index(os.path.join(ingestion_folder, "faiss_store"))
add_to_faiss(faiss, embs)

qdrant = get_qdrant_collection_local()
add_to_qdrant_local(qdrant, embs, texts)

# weaviate = get_weaviate_collection_local()
# add_to_weaviate_local(weaviate, embs, texts)

/Users/sunnyraj/code_files/git_repos/ragXplorer/rag_utils/indexing.py:161: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [ ]:
# ----------------------
# ✅ STEP 3: Time a query from each
query = "What is the topic of the text?"

# (example benchmarking code, to be replaced by actual metrics)
from time import time
start = time()
_ = chroma.query(query)
print("Chroma time:", time() - start)

# Repeat for FAISS, Qdrant, Weaviate